In [1]:
import timm
import torch
import timm.data
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torchvision.models as models
import torchvision.transforms as T
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"

canon_df = pd.read_csv(file_path)

canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

cols = ['ImageFilePath', 'Genus']

df = canon_df[cols]


In [3]:
df.head(2)


,ImageFilePath,Genus
0,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax
1,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax


In [4]:

model = timm.create_model('efficientnet_b3a', pretrained=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)


for param in model.parameters():
    param.requires_grad = False


preprocess_train = timm.data.create_transform(
    input_size=(224, 224),
    is_training=True,
)

preprocess_val = timm.data.create_transform(
    input_size=(224, 224),
    is_training=False,
)


transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name efficientnet_b3a to current efficientnet_b3.
  model = create_fn(


In [5]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model.forward_features(image)
    
    return features.cpu().numpy().flatten() 


In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df['ImageFilePath'])])

le = LabelEncoder()

y = le.fit_transform(df['Genus'])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


100%|██████████| 1622/1622 [01:19<00:00, 20.47it/s]


In [7]:
models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),   
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=42)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 81.23% | Prec: 77.90% | Rec: 81.23% | F1: 79.28% | Bal Acc: 53.19% | MCC: 0.5843
LogisticRegression        | Acc: 92.31% | Prec: 93.23% | Rec: 92.31% | F1: 92.22% | Bal Acc: 80.33% | MCC: 0.8503
SVMLinear                 | Acc: 93.54% | Prec: 93.79% | Rec: 93.54% | F1: 92.18% | Bal Acc: 66.68% | MCC: 0.8621


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 65.54% | Prec: 63.47% | Rec: 65.54% | F1: 52.60% | Bal Acc: 33.99% | MCC: 0.0989
MLP_Baseline              | Acc: 92.31% | Prec: 92.83% | Rec: 92.31% | F1: 91.42% | Bal Acc: 68.16% | MCC: 0.8354


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_NearestNeighbor,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax


In [9]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.812308,0.778984,0.812308,0.792819,0.531859,0.584278
LogisticRegression,LogisticRegression,0.923077,0.93235,0.923077,0.922182,0.803318,0.850316
SVMLinear,SVMLinear,0.935385,0.937934,0.935385,0.921756,0.666829,0.862102
NearestNeighbor,NearestNeighbor,0.655385,0.634658,0.655385,0.525981,0.339869,0.098856
MLP_Baseline,MLP_Baseline,0.923077,0.928316,0.923077,0.914172,0.681644,0.835381


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/50.EfficientNet-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/50.EfficientNet-linear-probing-species-metrics.csv", index=False)